<a href="https://colab.research.google.com/github/clammyface/8X8-dot-matrix-droplet-projects/blob/main/stocks_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost

In [ ]:
df2=pd.read_csv('Book1.csv')

In [ ]:
df2['6M return'] = df2['6M return'].astype(str).str.replace('%', '').astype(float)
df2['1Y return'] = df2['1Y return'].astype(str).str.replace('%', '').astype(float)
df2['2Y return'] = df2['2Y return'].astype(str).str.replace('%', '').astype(float)
df2['3Y return'] = df2['3Y return'].astype(str).str.replace('%', '').astype(float)
df2['5Y return'] = df2['5Y return'].astype(str).str.replace('%', '').astype(float)
df2['Predict']=df2['Predict'].astype(str).str.replace('%', '').astype(float)
df2

,PE,6M return,1Y return,2Y return,3Y return,5Y return,Predict,Month
0,19.96,132.9,73.7,29.7,9.7,13.4,13.7,4-Jan
1,19.74,76.1,69.3,25.6,10.0,12.9,16.8,4-Feb
2,20.70,56.3,81.1,25.2,15.6,10.4,14.9,4-Mar
3,18.68,33.3,92.3,28.7,16.9,12.9,5.9,4-Apr
4,11.94,-15.6,47.4,20.1,8.3,5.6,40.7,4-May
...,...,...,...,...,...,...,...,...
248,23.51,28.1,30.1,20.0,14.0,18.5,NaN,Sep-24
249,24.25,31.9,32.1,23.6,13.7,17.8,NaN,Oct-24
250,22.68,15.2,28.0,15.7,10.7,15.4,NaN,Nov-24
251,22.35,8.9,19.8,13.6,12.2,15.0,NaN,Dec-24


In [ ]:
df2.tail(14)
df2.drop(index=252,inplace=True)
df2.tail(12)


,PE,6M return,1Y return,2Y return,3Y return,5Y return,Predict,Month
240,23.18,26.6,19.5,11.1,15.8,15.0,NaN,Jan-24
241,22.38,20.9,23.2,11.1,15.0,15.0,NaN,Feb-24
242,23.06,32.1,28.0,16.0,14.8,14.0,NaN,Mar-24
243,23.02,32.3,29.1,12.7,14.7,13.8,NaN,Apr-24
244,22.03,42.3,24.8,15.2,15.7,13.7,NaN,May-24
245,22.13,31.7,25.8,18.7,14.3,14.0,NaN,Jun-24
246,22.97,23.3,24.9,23.8,15.5,15.3,NaN,Jul-24
247,23.45,32.9,26.7,20.1,16.3,17.9,NaN,Aug-24
248,23.51,28.1,30.1,20.0,14.0,18.5,NaN,Sep-24
249,24.25,31.9,32.1,23.6,13.7,17.8,NaN,Oct-24


In [ ]:
df3=df2.iloc[240:,:]
df3
df2.drop(index=df2.index[240:],inplace=True)

In [ ]:
df4=df2['Month']
print(type(df4))
print(df4)

<class 'pandas.core.series.Series'>
0       4-Jan
1       4-Feb
2       4-Mar
3       4-Apr
4       4-May
        ...  
235    Aug-23
236    Sep-23
237    Oct-23
238    Nov-23
239    Dec-23
Name: Month, Length: 240, dtype: object


In [ ]:
df2.drop(columns='Month',inplace=True)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Assuming df2 is your DataFrame
# Split the DataFrame into features (X) and target (y)
X = df2.drop(columns=['Predict'])  # Features
y = df2['Predict']                # Target column

# Normalize features (X) and target (y) using StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)  # Normalize features
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  # Normalize target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.25, random_state=42)

# Train the Support Vector Regressor (SVR) model
svr_model = SVR(kernel='rbf', C=30, epsilon=0.2)
svr_model.fit(X_train, y_train.ravel())  # Use ravel() to flatten y_train for SVR

# Predict on the test set
y_pred_scaled = svr_model.predict(X_test)

# Inverse transform to get original scale
y_test_original = scaler_y.inverse_transform(y_test)
y_pred_original = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))

# Create a DataFrame to compare actual and predicted values
results_df = pd.DataFrame({
    'Actual (Original)': y_test_original.flatten(),
    'Predicted (Original)': y_pred_original.flatten()
})

# Print the results DataFrame
print("Results DataFrame:")
print(results_df.head(20))  # Print first 20 rows

# Evaluate the model
mse = mean_squared_error(y_test_original, y_pred_original)
r2 = r2_score(y_test_original, y_pred_original)

# Print evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-Squared: {r2}")


Results DataFrame:
    Actual (Original)  Predicted (Original)
0                36.0             37.757327
1                41.7             44.234603
2                 5.5             16.252920
3                10.3              9.071815
4                 0.6             11.038599
5                11.1              8.805557
6                25.8             31.420034
7                -9.2             -4.509724
8                32.7             60.011059
9                 3.0              7.361900
10              -51.1            -51.412594
11               12.5             28.102803
12               34.5              7.609855
13               -1.8              4.066423
14               30.3              8.994393
15               28.0             18.902320
16              -18.7            -18.321326
17               87.0             81.125044
18               13.7             22.938804
19                9.9              5.725399
Mean Squared Error (MSE): 107.37895253425754
R-Squared: 0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Assuming df2 is your DataFrame
# Split the DataFrame into features (X) and target (y)
X = df2.drop(columns=['Predict'])  # Features
y = df2['Predict']                # Target column

# Normalize features (X) and target (y) for SVM
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)  # Normalize features
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  # Normalize target

# Train-test split
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X_scaled, y_scaled, test_size=0.25, random_state=42)

# Train Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train_rf, y_train_rf)

# Train Support Vector Regressor (SVM)
svr_model = SVR(kernel='rbf', C=35, epsilon=0.2)
svr_model.fit(X_train_svm, y_train_svm.ravel())

# Predict using both models
y_pred_rf = rf_model.predict(X_test_rf)  # Random Forest predictions
y_pred_svm_scaled = svr_model.predict(X_test_svm)  # SVM predictions (scaled)

# Inverse transform SVM predictions to the original scale
y_pred_svm = scaler_y.inverse_transform(y_pred_svm_scaled.reshape(-1, 1)).flatten()

# Combine predictions with weighted averaging and prioritize SVM for negative values
weight_rf = 0.7  # Weight for Random Forest
weight_svm = 0.3  # Weight for SVM

# Final predictions
y_pred_combined = np.where(
    y_pred_svm < 0,  # If SVM prediction is negative
    y_pred_svm,      # Use SVM prediction for negative cases
    (weight_rf * y_pred_rf + weight_svm * y_pred_svm)  # Weighted average otherwise
)

# Create a DataFrame to compare actual and combined predictions
results_df = pd.DataFrame({
    'Actual': y_test_rf.values,
    'Random Forest': y_pred_rf,
    'SVM': y_pred_svm,
    'Combined Prediction': y_pred_combined
})

# Print the results
print("Results DataFrame:")
print(results_df.head(20))  # Print the first 20 rows

# Evaluate the combined model
mse = mean_squared_error(y_test_rf, y_pred_combined)
r2 = r2_score(y_test_rf, y_pred_combined)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-Squared: {r2}")


Results DataFrame:
    Actual  Random Forest        SVM  Combined Prediction
0     36.0        27.9220  38.274302            31.545306
1     41.7        47.1480  43.405325            45.838064
2      5.5        14.3425  16.397875            15.061881
3     10.3        13.9130   9.029325            12.203714
4      0.6        11.9115  11.102033            11.628187
5     11.1         7.2395   8.762823             7.772663
6     25.8        20.7565  32.054748            24.710887
7     -9.2         3.1695  -4.181216            -4.181216
8     32.7        46.2205  60.919007            51.364977
9      3.0         7.1750   7.161117             7.170141
10   -51.1       -49.0590 -52.395731           -52.395731
11    12.5        14.7625  28.694096            19.638559
12    34.5        22.6230   7.111074            17.193826
13    -1.8         4.3980   4.779351             4.531473
14    30.3        20.6530   8.203011            16.295504
15    28.0        17.5915  18.745981            17.99